In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pickle
import MDAnalysis, sys
from MDAnalysis import transformations
%config InlineBackend.figure_format = 'retina'
dt=0.5# fs


/Users/sigbjobo/miniforge3/lib/python3.9/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [2]:
ice='IceV'

In [4]:
def extract_form_log(fn,dt):
    lines=open(fn,'r').readlines()
    start=np.where([('Step' in l) for l in lines])[-1][0]
    data={keyi.lower(): []  for keyi in lines[start].split()}
    for l in lines[start+1:]:
        ls=l.split()
        if ls[0].isdigit():
            for i, key in enumerate(data.keys()):
                data[key].append(float(ls[i]))
        else:
            break
    
    for key in data.keys():
        data[key]=np.array(data[key])
    data['time_ps']=dt*data['step']/1000
    return data

In [5]:
sims=dict()
for f in glob.glob('*_*/log.lammps'):
    try:
        sims[f.replace('/log.lammps','')]=extract_form_log(f,dt)
    except:
        pass

with open('1-bulk.pickle', 'wb') as handle:
    pickle.dump(sims, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('1-bulk.pickle', 'rb') as handle:
    sims = pickle.load(handle)

In [6]:
print(sims.keys())

dict_keys(['IceV_6000atm_265K', 'Liquid_5000atm_240K', 'IceV_5000atm_250K', 'IceV_6000atm_245K', 'IceV_5000atm_235K', 'IceV_6000atm_255K', 'IceV_5000atm_240K', 'IceV_6000atm_260K', 'Liquid_6000atm_250K', 'IceV_5000atm_255K', 'IceV_6000atm_250K', 'IceV_5000atm_245K'])


In [13]:
sims=dict()
with open('avg_{}_box.dat'.format(ice),'w') as fp:
    
    for fold in glob.glob(ice+'_*/'):
        #try:
        key=fold[:-1]
        ls=key.split('_')

        u=MDAnalysis.Universe(fold+'/water.data',fold+'/dump.dcd')
        dims=[]
        for ts in u.trajectory:
            dims.append(u.dimensions)
        p=round(float(ls[1].strip('atm')),0)
        t=round(float(ls[2].strip('K')),0)    
        dims=np.array(dims)[len(dims)//10:].mean(axis=0)
        fp.write('\t'.join(['{}'.format(d) for d in [p,t]+dims.tolist()])+'\n')

In [14]:
!cat avg_IceV_box.dat

6000.0	265.0	18.092849731445312	22.672405242919922	20.540794372558594	89.86395263671875	109.45672607421875	90.08244323730469
5000.0	250.0	18.116540908813477	22.680185317993164	20.57480239868164	89.90642547607422	109.6020278930664	90.07376098632812
6000.0	245.0	18.0606689453125	22.614473342895508	20.516645431518555	89.88121795654297	109.64061737060547	90.07563781738281
5000.0	235.0	18.094995498657227	22.641199111938477	20.559104919433594	89.87788391113281	109.67622375488281	90.07684326171875
6000.0	255.0	18.085052490234375	22.644744873046875	20.534584045410156	89.8830795288086	109.54635620117188	90.0645980834961
5000.0	240.0	18.1019287109375	22.65399169921875	20.572790145874023	89.88616943359375	109.6635971069336	90.0670166015625
6000.0	260.0	18.08629608154297	22.646623611450195	20.54138946533203	89.87895202636719	109.53627014160156	90.06652069091797
5000.0	255.0	18.125165939331055	22.69745445251465	20.581453323364258	89.90628814697266	109.5675277709961	90.05548095703125
6000.0	250.0	18